In [1]:
# https://github.com/Dean0371/Tensorflow2.0/blob/225eb6f56a1df56b101b546a694f4825a8e0429e/Tensorflow2.0_%E8%B0%B7%E6%AD%8C/3_API/tf_function_and_auto_graph.py

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os,sys,time
import tensorflow as tf
from tensorflow import keras

print((tf.__version__))
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
matplotlib 2.2.2
numpy 1.14.5
pandas 0.23.0
sklearn 0.19.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
# tf.function and auto-graph
# python函数
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z>=0?scale *z :scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))   #   常量
print(scaled_elu(tf.constant([-3.,-2.5])))  # 列表向量  都何以全部接受并处理

# tensorflow的图函数
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))   #   常量
print(scaled_elu_tf(tf.constant([-3.,-2.5])))  # 列表向量  都何以全部接受并处理

print(scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True


In [3]:
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

22.4 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.1 ms ± 614 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
# 1 + 1/2 + 1/2^2 + ... + 1/2^n

def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    
    return total
print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [5]:
def display_tf_code(func):
    # 中间代码
    # python代码转为tf代码, 图就是通过此转换的
    code = tf.autograph.to_code(func)   # 还有to_graph 是将代码转为图的
    from IPython.display import display,Markdown
    display(Markdown('```python\n{}\n```'.format(code)))
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [6]:
display_tf_code(converge_to_2)

```python
def tf__converge_to_2(n_iters):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('converge_to_2', 'converge_to_2_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as converge_to_2_scope:
    total = ag__.converted_call(tf.constant, converge_to_2_scope.callopts, (0.0,), None, converge_to_2_scope)
    increment = ag__.converted_call(tf.constant, converge_to_2_scope.callopts, (1.0,), None, converge_to_2_scope)

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(iterates, total, increment):
      _ = iterates
      total += increment
      increment /= 2.0
      return total, increment
    total, increment = ag__.for_stmt(ag__.converted_call(range, converge_to_2_scope.callopts, (n_iters,), None, converge_to_2_scope), None, loop_body, get_state, set_state, (total, increment), ('total', 'increment'), ())
    do_return = True
    retval_ = converge_to_2_scope.mark_return_value(total)
  do_return,
  return ag__.retval(retval_)

```

In [9]:
# python ----->图
#   法一
def scaled_elu(z,scale=1.0,alpha=1.0):   # python函数
    # z>=0?scale *z :scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z,0.0)
    return scale * tf.where(is_positive,z,alpha * tf.nn.elu(z))
print(scaled_elu(tf.constant(-3)))   #   常量
print(scaled_elu(tf.constant([-3,-2.5])))  # 列表向量  都何以全部接受并处理
scaled_elu_tf = tf.function(scaled_elu)
scaled_elu_tf.python_function  # 返回原来的python函数
print(scaled_elu(tf.constant(-3)))
print(scaled_elu(tf.constant([-3,-2.5]))) # 与上边结果相同,转换的作用是速度加快
#   法二
#   1+1/2+....1/2^n
@tf.function
def converge_to(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total
print(converge_to(20.))

# def display_tf_code(func):
#     # 中间代码
#     # python代码转为tf代码, 图就是通过此转换的
#     code = tf.autograph.to_code(func)   # 还有to_graph 是将代码转为图的
#     from IPython.display import display,Markdown
#     display(Markdown('```python\n{}\n```'.format(code)))
# display_tf_code(scaled_elu)

TypeError: Cannot convert 0.0 to EagerTensor of dtype int32

In [10]:
# 变量参与图结构，要先进行定义初始化。若var在内部会报错, 神经网络中大多是变量,需要在外边初始化
var = tf.Variable(0.)

@tf.function
def add_21():
    return var.assign_add(21) # +=

print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [11]:
@tf.function
def cube(z):
    return tf.pow(z, 3)

print(cube(tf.constant([1,2,3])))
print(cube(tf.constant([1.,2.,3.])))

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)
tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)


In [12]:
@tf.function(input_signature=[tf.TensorSpec([None],tf.int32,name='x')])
def cube(z):  # 可接收浮点数,整数  使用输入签名后会限制只能输入int32
    return tf.pow(z,3)

try:
    print(cube(tf.constant([1.,2.,3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1,2,3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [13]:
# 只有经过输入签名的才能保存为Saved_Model,在这个过程中使用get_concrete_function,把tf.function标注的转换为有图建议的函数
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None],tf.int32))

print(cube_func_int32)

In [16]:
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([2],tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5],tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3])))

True
True
True


In [18]:
# @tf.function py func -> tf graph
# get_concrete_function -> add input signature -> saved model

cube_func_int32.graph  # 图

In [19]:
cube_func_int32.graph.get_operations()  # 获取操作

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [20]:
cube_func_int32.graph.as_graph_def()  # 显示图的结构信息

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}

In [ ]:
# Identity节点一般是过渡op，输入是什么，输出就是什么